In [111]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timezone, timedelta
from zoneinfo import ZoneInfo
import re
import numpy as np

In [3]:
tal = pd.read_csv('episode_info', index_col=False)
#tal.drop("Unnamed: 0", axis=1, inplace=True)
#tal.drop_duplicates(inplace=True) 

In [4]:
#tal.to_csv('episode_info', index=False)

In [5]:
tal.isna().sum()

ep_num            0
ep_name           0
pub_date          0
host              0
contributers      0
num_acts          0
last_timestamp    0
url_suffix        0
dtype: int64

In [6]:
#tal['url_suffix'] = tal.apply(lambda row: str(row['ep_num']) + '/' + row['ep_name'].replace(' ','-'), axis=1)
tal.head()

,ep_num,ep_name,pub_date,host,contributers,num_acts,last_timestamp,url_suffix
0,1,New Beginnings,2017-12-14 10:24:29-08:00,Ira Glass,"['Kevin Kelly', 'Lawrence Steger', 'Ira Glass']",4,0 days 01:01:50.290000,1/New-Beginnings
1,2,Small Scale Sin,2017-12-14 10:24:29-08:00,Ira Glass,"['Announcer', 'Ira Glass']",3,0 days 01:01:13.840000,2/Small-Scale-Sin
2,3,Poultry Slam 1995,2017-12-14 10:24:29-08:00,Ira Glass,"['David Sedaris', 'Chickenman Announcer', 'Ira...",4,0 days 00:58:57.870000,3/Poultry-Slam-1995
3,4,Vacations,2017-12-14 10:24:29-08:00,Ira Glass,"['Sandra Tsing Loh', 'Ira Glass', 'David Sedar...",3,0 days 00:58:42.070000,4/Vacations
4,5,Anger and Forgiveness,2017-12-14 10:24:29-08:00,Ira Glass,"['Glen Fitzgerald', 'Jack Hitt', 'Cheryl Trykv...",5,0 days 01:00:33.100000,5/Anger-and-Forgiveness


### Let's work on data validation from the episode pages now that we have the episode number and name

for i in np.arange(824,853):
    ep_number = str(i)
    transcript_url = 'https://www.thisamericanlife.org/'+ ep_number + '/transcript'
    page = requests.get(transcript_url)
    with open('transcript-html/'+ep_number+'.html','wb+') as f: 
        f.write(page.content)

domain = 'https://www.thisamericanlife.org/'
for ep in tal.itertuples():
    url = domain + ep[8]
    ep_num = ep[1]
    page = requests.get(url)
    with open('episode-html/'+str(ep_num)+'.html','wb+') as f:
        f.write(page.content)
    f.close()
    #print(url, ep_num)

In [7]:
tal

,ep_num,ep_name,pub_date,host,contributers,num_acts,last_timestamp,url_suffix
0,1,New Beginnings,2017-12-14 10:24:29-08:00,Ira Glass,"['Kevin Kelly', 'Lawrence Steger', 'Ira Glass']",4,0 days 01:01:50.290000,1/New-Beginnings
1,2,Small Scale Sin,2017-12-14 10:24:29-08:00,Ira Glass,"['Announcer', 'Ira Glass']",3,0 days 01:01:13.840000,2/Small-Scale-Sin
2,3,Poultry Slam 1995,2017-12-14 10:24:29-08:00,Ira Glass,"['David Sedaris', 'Chickenman Announcer', 'Ira...",4,0 days 00:58:57.870000,3/Poultry-Slam-1995
3,4,Vacations,2017-12-14 10:24:29-08:00,Ira Glass,"['Sandra Tsing Loh', 'Ira Glass', 'David Sedar...",3,0 days 00:58:42.070000,4/Vacations
4,5,Anger and Forgiveness,2017-12-14 10:24:29-08:00,Ira Glass,"['Glen Fitzgerald', 'Jack Hitt', 'Cheryl Trykv...",5,0 days 01:00:33.100000,5/Anger-and-Forgiveness
...,...,...,...,...,...,...,...,...
845,848,The Official Unofficial Record,2024-11-23 08:50:53-08:00,Ira Glass,"['Lilly Sullivan', 'Nancy Updike', 'Ira Glass']",3,0 days 00:59:44.810000,848/The-Official-Unofficial-Record
846,849,The Narrator,2024-12-14 03:40:47-08:00,Chana Joffe-Walt,['Chana Joffe-Walt'],2,0 days 00:55:52.570000,849/The-Narrator
847,850,"If You Want to Destroy My Sweater, Hold This T...",2024-12-21 02:30:34-08:00,Ira Glass,"['Lilly Sullivan', 'Mike Comite', 'Diane Wu', ...",3,0 days 01:02:52.080000,"850/If-You-Want-to-Destroy-My-Sweater,-Hold-Th..."
848,851,Try a Little Tenderness,2025-01-13 07:30:23-08:00,Ira Glass,"['Etgar Keret', 'Aviva DeKornfeld', 'Ira Glass']",4,0 days 00:58:06.240000,851/Try-a-Little-Tenderness


* pub date is off for older episodes (and some newer episodes?)
* some episode names are truncated?
* some contributers are off? 

In [14]:
test = open('episode-html/30.html', 'r')
soup = BeautifulSoup(test, 'html.parser')
date = soup.find("meta", property="article:published_time")['content']
dt = datetime.strptime(date, "%Y-%m-%dT%H:%M:%S%z")
dt#.astimezone(ZoneInfo('US/Pacific'))

datetime.datetime(1996, 7, 26, 0, 0, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=72000)))

In [30]:
def get_episode(zup): 
    try:
        return re.split(" - ", zup.title.text)[0]
    except Exception as e: 
        print(e)
        return None
        
def get_date(zup): 
    try:
        date = soup.find("meta", property="article:published_time")['content']
        dt = datetime.strptime(date, "%Y-%m-%dT%H:%M:%S%z")
        return dt
    except Exception as e: 
        print(e)
        return None

In [31]:
get_episode(soup)

'Obsession'

In [32]:
get_date(soup)

datetime.datetime(1996, 7, 26, 0, 0, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=72000)))

In [51]:
soup.find_all("div", class_='field field-name-field-contributor field-type-entityreference field-label-above')

[<div class="field field-name-field-contributor field-type-entityreference field-label-above"><h2 class="field-label">By</h2><div class="field-items"><div class="field-item even"><a href="/archive?contributor=8731">Ira Glass</a></div><div class="field-item odd"><a href="/archive?contributor=9085">Paul Tough</a></div></div></div>,
 <div class="field field-name-field-contributor field-type-entityreference field-label-above"><h2 class="field-label">By</h2><div class="field-items"><div class="field-item even"><a href="/archive?contributor=9045">Lauren Slater</a></div><div class="field-item odd"><a href="/archive?contributor=9085">Paul Tough</a></div></div></div>]

In [109]:
def get_contributors(soup):
    try:
        contributors = set()
        c_list = soup.find_all("div", class_='field field-name-field-contributor field-type-entityreference field-label-above')
        for i in c_list:
            for j in i.find_all('a'):
                contributors.add(j.text)
        return list(contributors)
    except Exception as e:
        return None


get_contributors(soup)



['Paul Tough', 'Lauren Slater', 'Ira Glass']

In [184]:
def get_acts(soup, ep_num):
    act_names = []
    r_href = re.compile(str(ep_num))
    try:
        for element in soup.find_all('a', class_='goto goto-act'):
            if re.search(r_href, str(element)):
                act_names.append(element.text)
        return act_names
    except Exception as e:
        return None

In [185]:
get_acts(soup, 30)

['Act One', 'Further Out', 'Sacred Versus Profane', 'Heroic Obsessions']

In [177]:
meow = soup.find_all("a", class_="goto goto-act", href="30")
meow

[]

In [114]:
tags = [tag.name for tag in soup.find_all()]
class_list = set()
for tag in tags: 
    for i in soup.find_all(tag):
        if i.has_attr("class"):
            if len(i['class']) !=0:
                class_list.add(" ".join(i['class']))

In [115]:
class_list

{'about has-children',
 'act-header',
 'actions',
 'acts',
 'apple',
 'block block-system',
 'body',
 'bottom',
 'button form-submit',
 'clear',
 'close',
 'close ignore',
 'collection',
 'column sidebar',
 'contact',
 'container clearfix',
 'content',
 'contributor',
 'copyright',
 'credit',
 'cta_link',
 'current',
 'date-display-single',
 'description',
 'desktop',
 'download',
 'element-invisible',
 'element-invisible element-focusable',
 'episode',
 'episode-header',
 'episode-title',
 'episodes',
 'facebook',
 'field field-name-body field-type-text-with-summary field-label-hidden',
 'field field-name-field-act-label field-type-text field-label-hidden',
 'field field-name-field-acts field-type-entityreference field-label-hidden',
 'field field-name-field-contributor field-type-entityreference field-label-above',
 'field field-name-field-episode-number',
 'field field-name-field-episode-number field-type-number-integer field-label-hidden',
 'field field-name-field-radio-air-date',


In [189]:
max_ep = 20
data = []

for ep in np.arange(1,max_ep+1):
    try:
        with open('episode-html/' + str(ep) + '.html', 'r') as f:
            content = f.read()
            s = BeautifulSoup(content, 'html.parser')
            name = get_episode(s)
            c = get_contributors(s)
            date = get_date(s)
            acts = get_acts(s, ep)
            data.append([ep, name, date, c, acts])
    except Exception as e:
        print(str(e) + " at episode " + str(ep))
    finally:
        f.close()

In [191]:
temp = pd.DataFrame(data, columns = ['num', 'name', 'pub_date', 'contributors', 'acts'])
temp[1:10]

,num,name,pub_date,contributors,acts
1,2,Small Scale Sin,1996-07-26 00:00:00-04:00,"[Jeff Dorchen, Ira Glass]","[Prologue, Act One, Act Two, Act Three, Act On..."
2,3,Poultry Slam 1995,1996-07-26 00:00:00-04:00,"[Julie Showalter, David Sedaris, Ira Glass]","[Act One, Act Two, Act Three, Act Four, Act Fi..."
3,4,Vacations,1996-07-26 00:00:00-04:00,"[Sandra Loh, Ira Glass, David Sedaris]","[Act One, Act Two, Act Three]"
4,5,Anger and Forgiveness,1996-07-26 00:00:00-04:00,"[Iris Moore, Larry Steeger, Jack Hitt, Ira Glass]","[Prologue, Act One, Act Two, Act Three, Act Fo..."
5,6,Christmas,1996-07-26 00:00:00-04:00,"[The Pinetree Gang, Beau O'Reilly, Ira Glass, ...","[Prologue, Act One, Act Two, Act Three, Act Tw..."
6,7,Quitting,1996-07-26 00:00:00-04:00,"[Lisa Buscani, Sandra Loh, Ira Glass]","[Prologue, Act One, Act Two, Act Three, Act Fo..."
7,8,New Year,1996-07-26 00:00:00-04:00,[Ira Glass],"[Prologue, Act One, Act Two, Act Three]"
8,9,Julia Sweeney,1996-07-26 00:00:00-04:00,"[Julia Sweeney, Ira Glass]","[Prologue, Julia's Brother Gets Cancer, Julia ..."
9,10,Double Lives,1996-07-26 00:00:00-04:00,"[Ira Glass, Neo-futurists]","[Prologue, The Book Tour, Dad In The Closet, L..."


In [192]:
tal

,ep_num,ep_name,pub_date,host,contributers,num_acts,last_timestamp,url_suffix
0,1,New Beginnings,2017-12-14 10:24:29-08:00,Ira Glass,"['Kevin Kelly', 'Lawrence Steger', 'Ira Glass']",4,0 days 01:01:50.290000,1/New-Beginnings
1,2,Small Scale Sin,2017-12-14 10:24:29-08:00,Ira Glass,"['Announcer', 'Ira Glass']",3,0 days 01:01:13.840000,2/Small-Scale-Sin
2,3,Poultry Slam 1995,2017-12-14 10:24:29-08:00,Ira Glass,"['David Sedaris', 'Chickenman Announcer', 'Ira...",4,0 days 00:58:57.870000,3/Poultry-Slam-1995
3,4,Vacations,2017-12-14 10:24:29-08:00,Ira Glass,"['Sandra Tsing Loh', 'Ira Glass', 'David Sedar...",3,0 days 00:58:42.070000,4/Vacations
4,5,Anger and Forgiveness,2017-12-14 10:24:29-08:00,Ira Glass,"['Glen Fitzgerald', 'Jack Hitt', 'Cheryl Trykv...",5,0 days 01:00:33.100000,5/Anger-and-Forgiveness
...,...,...,...,...,...,...,...,...
845,848,The Official Unofficial Record,2024-11-23 08:50:53-08:00,Ira Glass,"['Lilly Sullivan', 'Nancy Updike', 'Ira Glass']",3,0 days 00:59:44.810000,848/The-Official-Unofficial-Record
846,849,The Narrator,2024-12-14 03:40:47-08:00,Chana Joffe-Walt,['Chana Joffe-Walt'],2,0 days 00:55:52.570000,849/The-Narrator
847,850,"If You Want to Destroy My Sweater, Hold This T...",2024-12-21 02:30:34-08:00,Ira Glass,"['Lilly Sullivan', 'Mike Comite', 'Diane Wu', ...",3,0 days 01:02:52.080000,"850/If-You-Want-to-Destroy-My-Sweater,-Hold-Th..."
848,851,Try a Little Tenderness,2025-01-13 07:30:23-08:00,Ira Glass,"['Etgar Keret', 'Aviva DeKornfeld', 'Ira Glass']",4,0 days 00:58:06.240000,851/Try-a-Little-Tenderness
